In [12]:
from google.colab import drive
import numpy as np
import os
from os import listdir
from os.path import isfile, join, splitext
from PIL import Image, ImageOps
import numpy as np
import random
import math
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from keras.regularizers import l1
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras import Input
from matplotlib import pyplot

'''try:
    from google.colab import drive
    print('Running in colab...\n===================')
    COLAB = True
    # drive_home = '/content/drive'
    # drive.mount(drive_home)
    # print('Drive connected!\n================')
    !pip install madmom # torch==1.4.0 torchvision==0.5.0 --upgrade
    print('Installed dependencies!\n=======================')
    print('Downloading data...\n===================')
    if not os.path.exists('dataset.npz'):
      !wget -O dataset.npz "http://www.ifs.tuwien.ac.at/~vogl/other/dataset.npz"
    print('===================\nMake sure you activated GPU support: Edit->Notebook settings->Hardware acceleration->GPU\n==================')
except:
    print('=======================\nNOT running in colab...\n=======================')
    COLAB = False'''


'try:\n    from google.colab import drive\n    print(\'Running in colab...\n===================\')\n    COLAB = True\n    # drive_home = \'/content/drive\'\n    # drive.mount(drive_home)\n    # print(\'Drive connected!\n================\')\n    !pip install madmom # torch==1.4.0 torchvision==0.5.0 --upgrade\n    print(\'Installed dependencies!\n=======================\')\n    print(\'Downloading data...\n===================\')\n    if not os.path.exists(\'dataset.npz\'):\n      !wget -O dataset.npz "http://www.ifs.tuwien.ac.at/~vogl/other/dataset.npz"\n    print(\'===================\nMake sure you activated GPU support: Edit->Notebook settings->Hardware acceleration->GPU\n==================\')\nexcept:\n    print(\'=======================\nNOT running in colab...\n=======================\')\n    COLAB = False'

In [5]:
#If you didn't run the previous code that do the preprocessing -> run this part of code please
#DATASET USED FOR THE MODEL FOR TEMPO ESTIMATION
!gdown --id 1laAPu4rWL--uQqImUVLRucxX45WioDo6
!gdown --id 1dss_W5QL0y-C3LxuCHWO72FnXOq1nc3y

path_images=["Image_spec_t/"]
path_ann=["Ann_bpm/"]
from zipfile import ZipFile
for p in path:
  with ZipFile(p, 'r') as zipp:
      # printing all the contents of the zip file
      #zip.printdir()
      print('Extracting all the files now...')
      zipp.extractall()
      print('Done!')
flag=0

Downloading...
From: https://drive.google.com/uc?id=1laAPu4rWL--uQqImUVLRucxX45WioDo6
To: /content/Image_spec_t.zip
100% 264M/264M [00:01<00:00, 199MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dss_W5QL0y-C3LxuCHWO72FnXOq1nc3y
To: /content/Ann_bpm.zip
100% 1.17M/1.17M [00:00<00:00, 30.0MB/s]
Extracting all the files now...
Done!
Extracting all the files now...
Done!


In [ ]:
#If you have done the preprocessing using the previous notebook, please run this part of code
path_images=["Dataset/Tempo/GS/Image_spec","Dataset/Tempo/EXT/Image_spec"]
path_ann=["Dataset/Tempo/GS/Ann","Dataset/Tempo/EXT/Ann"]
flag=1

In [21]:
def split_train_test_validation(path_label,path_img):
  f_l=[]
  f_i=[]
  for i,l in zip(path_img,path_label):
    f_l.extend([os.path.join(root, name) for root, dirs, files in os.walk(l) for name in files if name.endswith((".txt"))])
    #print(splitext(f_l[0])[0].split('/'))
    f_i.extend([os.path.join(root, name) for root, dirs, files in os.walk(i) for name in files if name.endswith((".jpeg"))])
    
    f_l=[os.path.join(f.split('/')[1],f.split('/')[2]) for f in f_l]
    f_i=[os.path.join(f.split('/')[1],f.split('/')[2]) for f in f_i]
    
  f_l= np.asarray(f_l)
  f_i= np.asarray(f_i)
  print("Sorting both images and annotations...")
  for n in range(f_l.shape[0]):
    while True:
      s=splitext(f_l[n])[0]
      if "lerch" in s:
        s=s[:len(s)-6]
      res=np.where(f_i==f"{str(s)}.jpeg")
      if(n==res[0]):
        break
      f_i[n], f_i[res[0][0]] = f_i[res[0][0]], f_i[n]
  c=list(zip(f_l,f_i))
  random.shuffle(c)
  f_l,f_i=zip(*c)
  images=[]
  ann=[]
  print("Processing the images...")
  for f in f_i:
    images.append(np.asarray(ImageOps.grayscale(Image.open(f"Image_spec_t/{f}").resize((40,512)))))
  print("Processing the annotations")
  for an in f_l:
    with open(f"Ann_bpm/{an}") as f:
      bpm=int(float(f.readlines()[0]))
      ann.append(bpm)

  min_annotation=min(ann)
  while min_annotation==0:
    ann[ann.index(min_annotation)]=130
    min_annotation=min(ann)
  MIN_BPM=min_annotation
  max_annotation=max(ann)-min(ann)
  print(f"Minimum bpm found is {min_annotation}, the maximum is {max_annotation}")
  ann=[x-min_annotation  for x in ann]
  test_x=np.asarray(images[0:math.ceil(len(f_i)*0.2)])/255
  test_file=f_i[:math.ceil(len(f_i)*0.2)]
  test_y=np.array(ann[0:math.ceil(len(f_i)*0.2)])
  train_x=np.asarray(images[len(test_x):len(test_x)+math.ceil(len(f_i)*0.6)])/255
  train_y=np.array(ann[len(test_x):len(test_x)+math.ceil(len(f_i)*0.6)])
  val_x=np.asarray(images[len(test_x)+len(train_x):])/255
  val_y=np.array(ann[len(test_x)+len(train_x):])
  train_y = to_categorical(train_y, max_annotation+1)
  test_y = to_categorical(test_y, max_annotation+1) 
  print(max_annotation)
  print(val_y)
  val_y= to_categorical(val_y, max_annotation+1)
  print(f"Split a total of {len(f_i)} samples into three sets with {len(train_x)} training, {len(val_x)} validation and {len(test_x)} test samples.")
  return test_x,test_y,train_x,train_y,val_x,val_y,test_file,MIN_BPM


x_test,y_test,x_train,y_train,x_val,y_val,test_file,MIN_BPM=split_train_test_validation(path_ann,path_images)


['Image_spec_t/Giantstep/1896782.LOFI_4.jpeg', 'Image_spec_t/Giantstep/4409752.LOFI_1.jpeg', 'Image_spec_t/Giantstep/5355157.LOFI_2.jpeg', 'Image_spec_t/Giantstep/4017909.LOFI_3.jpeg', 'Image_spec_t/Giantstep/5076821.LOFI_4.jpeg', 'Image_spec_t/Giantstep/4540818.LOFI_1.jpeg', 'Image_spec_t/Giantstep/5347155.LOFI_2.jpeg', 'Image_spec_t/Giantstep/3173803.LOFI_2.jpeg', 'Image_spec_t/Giantstep/210560.LOFI_2.jpeg', 'Image_spec_t/Giantstep/4101363.LOFI_2.jpeg', 'Image_spec_t/Giantstep/4226434.LOFI_3.jpeg', 'Image_spec_t/Giantstep/4486412.LOFI_3.jpeg', 'Image_spec_t/Giantstep/2061919.LOFI_3.jpeg', 'Image_spec_t/Giantstep/4486124.LOFI_4.jpeg', 'Image_spec_t/Giantstep/5214265.LOFI_1.jpeg', 'Image_spec_t/Giantstep/5051606.LOFI_2.jpeg', 'Image_spec_t/Giantstep/3372883.LOFI_3.jpeg', 'Image_spec_t/Giantstep/3829043.LOFI_2.jpeg', 'Image_spec_t/Giantstep/1968659.LOFI_4.jpeg', 'Image_spec_t/Giantstep/5314439.LOFI_1.jpeg', 'Image_spec_t/Giantstep/4017781.LOFI_3.jpeg', 'Image_spec_t/Giantstep/4140514.LO

In [13]:
model = tf.keras.Sequential([
    layers.Conv2D(16, (5, 5), activation='relu', input_shape=(512, 40,1)),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (5, 5), activation='relu'), 
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (5, 5), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    tf.keras.layers.Reshape(target_shape=(-1,32), name='reshape'),           
    layers.Bidirectional(layers.LSTM(32, return_sequences=True,activation='tanh')),
    layers.Bidirectional(layers.LSTM(32, return_sequences=True,activation='tanh')),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(153,activation='softmax')
    
])
model.build((512,40))
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.055),loss='categorical_crossentropy',metrics=['acc'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
history = model.fit(x_train, y_train, epochs=75, batch_size=99,validation_data=(x_val, y_val))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 508, 36, 16)       416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 254, 18, 16)      0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 254, 18, 16)      64        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 250, 14, 32)       12832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 125, 7, 32)       0         
 2D)                                                             
                                                        

KeyboardInterrupt: ignored

In [ ]:
pyplot.plot(history.history['acc'], label='train')
pyplot.plot(history.history['val_acc'], label='test')

In [32]:

def get_accuracy(model, true_x, true_y): 
    res = model.predict(true_x)
    res = np.argmax(res, axis=-1)
    acc0 = 0
    acc1 = 0
    acc2 = 0
    tol=0.08
    for i in range(len(true_y)):
        r=res[i]
        t=true_y[i]
        if r>t-1 and r<t+1:
          acc0+=1
          acc1+=1
          acc2+=1
        if r>t-(t*tol) and r<t+(t*tol):
          acc1+=1
          acc2+=1
        elif (r>2*t-(t*tol) and r<2*t+(t*tol)) or (r>t//2-(t*tol) and r<t//2+(t*tol)):
          acc2+=1
        else:
            pass
    tot = len(true_y)
    print('acc0 : {}'.format((acc0/tot)))
    print('acc1 : {}'.format((acc1/tot)))
    print('acc2 : {}'.format((acc2/tot)))

def get_dataset(label,img,test_file):
  f_l=[]
  f_i=[]
  for i,l in zip(img,label):
    f_l.extend([os.path.join(root, name) for root, dirs, files in os.walk(l) for name in files if name.endswith((".txt"))])
    f_i.extend([os.path.join(root, name) for root, dirs, files in os.walk(i) for name in files if name.endswith((".jpeg"))])
  f_l= np.asarray(f_l)
  f_i= np.asarray(f_i)
  for n in range(f_l.shape[0]):
    while True:
      s=splitext(f_l[n])[0]
      if "lerch" in s:
        s=s[:len(s)-6]
      res=np.flatnonzero(np.core.defchararray.find(f_i,f"{str(s).split('/')[-1]}")!=-1)
      if(n==res[0]):
        break
      f_i[n], f_i[res[0]] = f_i[res[0]], f_i[n]
  c=list(zip(f_l,f_i))
  giant_i=[]
  ext_i=[]
  ext_l=[]
  giant_l=[]
  lofi="LOFI"
  
  for f in f_i:
    if f in test_file: 
      if lofi in f:
        idx=np.where(f_i == f)
        giant_i.append(f)
        giant_l.append(f_l[idx])
      else:
        idx=np.where(f_i == f)
        ext_i.append(f)
        ext_l.append(f_l[idx])
  e_i=[]
  e_l=[]
  gi_i=[]
  gi_l=[]
  for g1 in ext_i:
    e_i.append(np.asarray(ImageOps.grayscale(Image.open(f"Image_spec_t/{g1}").resize((40,512))))/255)
  for g2 in giant_i:  
    gi_i.append(np.asarray(ImageOps.grayscale(Image.open(f"Image_spec_t/{g2}").resize((40,512))))/255)
  for g1 in ext_l:
    with open(f"Ann_bpm/{g1[0]}") as f:
      e_l.append(int(float(f.readlines()[0])))
  for g2 in giant_l:
    with open(f"Ann_bpm/{g2[0]}") as f:
      gi_l.append(int(float(f.readlines()[0])))
  e_i=np.asarray(e_i)
  e_l=np.asarray(e_l)
  gi_i=np.asarray(gi_i)
  gi_l=np.asarray(gi_l)
  return e_i,e_l,gi_i,gi_l
  
e_i,e_l,giant_i,giant_l=get_dataset(path_ann,path_images,test_file)
get_accuracy(model,e_i,e_l)

ValueError: ignored